turn the Jaspar experiments  https://github.com/TomConlin/Jaspar_FA into RDF to load

In [1]:
import csv
import hashlib

the triples to generate may be found by issueing

```awk -F'"' '/.* -> .*/ {print "# <" $2 "><" $6 "><" $4 ">"}' jaspar_target_model.gv```

In [2]:
# <NCBIGene:123><SO:adjacent_to><BNODE:gene1_upstream_region>
# <BNODE:gene1_upstream_region><rdfs:label><gene1_upstream_region>
# <BNODE:gene1_upstream_region><GENO:has_extent><1000 (region extent bp)>
# <BNODE:gene1_upstream_region><rdf:type><SO:five_prime_flanking_region>
# <BNODE:gene1_upstream_region><rdf:comment><Candidate SO:TF_binding_sites>
# <BNODE:motif_set><rdfs:label><motif_set_sig>
# <BNODE:motif_set><rdf:type><SIO:collection>
# <BNODE:motif_set><rdf:value><3 (flavors)>
# <BNODE:motif_set><OIO:hasdbxref><http:JASPAR:motif>
# <BNODE:motif_set><OIO:subset><BNODE:motif_set>
# <BNODE:motif_set><RO:has member><JASPAR:motif>
# <JASPAR:motif><rdf:type><SO:nucleotide_motif>
# <JASPAR:motif><OIO:hasdbxref><http:JASPAR:motif>
# <JASPAR:motif><rdfs:label><motif>
# <BNODE:gene1_upstream_region><RO:member of><BNODE:pairwise similarity>
# <BNODE:gene2_upstream_region><RO:member of><BNODE:pairwise similarity>
# <BNODE:pairwise similarity><rdfs:label><gene1_region gene2_region>
# <BNODE:pairwise similarity><SWO:Similarity score><0.73>
# <BNODE:pairwise similarity><rdf:type><SWO:Jaccard’s index>


In [3]:
# return a deterministic digest of input
# the 'b' is an experiment forcing the first char to be
# non numeric but valid hex
# which is in no way required for RDF
# but can help when using the identifier in other contexts
def digest_id(wordage):
    return 'b' + hashlib.sha1(wordage.encode('utf-8')).hexdigest()[1:20]

In [4]:
# stand in till I expand the curises to full IRI
def write_triple(sub, prd, obj):
    triples.append(sub + ' ' + prd + ' ' + obj + ' .')


In [5]:
regions = {    
    '1k' : {
        'f1' : 'gene_motifsetsig_1k.tab'
    }, 
    '2k' : {
        'f1' : 'gene_motifsetsig_1k.tab'
    },
    '5k' : {
        'f1' : 'gene_motifsetsig_1k.tab'
    }
}

The triples to produce form these files are:

    # <NCBIGene_123><SO:adjacent_to><BNODE:gene_upstream_region>
    # <BNODE:gene_upstream_region><rdfs:label><gene_upstream_region>
    # <BNODE:gene_upstream_region><GENO:has_extent><1000 (region extent bp)>
    # <BNODE:gene_upstream_region><rdf:type><SO:five_prime_flanking_region>
    # <BNODE:gene_upstream_region><rdf:comment><Candidate SO:TF_binding_sites>
    # <BNODE:motif_set><rdfs:label><motif_set_sig>
    # <BNODE:motif_set><rdf:type><SIO:collection>


In [6]:
triples = []

for extent in regions:
    # print(extent)
    # print(regions[extent])
    for fname in regions[extent]:
       # print(fname)
       # print(regions[extent][fname])    
        with open(regions[extent][fname], 'r') as tabfile:
            filereader = csv.reader(tabfile, delimiter='\t')
            line_counter = 0
            for row in filereader:
                line_counter += 1     
                (geneid, centroid, motifsetsig, count) = row
                gene = 'NCBIGene:' + str(geneid)
                region_label = gene + "_up"  + extent + "_@" + centroid
                region = '_:' + digest_id(region_label)
                motifset = '_:' + digest_id(motifsetsig)
    
                # using the triple templates extracted from the target model
    
                # <NCBIGene_123><SO:adjacent_to><BNODE:gene_upstream_region>
                write_triple(gene, 'SO:adjecent_to', region)
                # <BNODE:gene_upstream_region><rdfs:label><gene_upstream_region>
                write_triple(region, 'rdfs:label', region_label)
                # <BNODE:gene_upstream_region><GENO:has_extent><1000>
                write_triple(region, 'GENO:has_extent', extent)
                # <BNODE:gene_upstream_region><rdf:type><SO:five_prime_flanking_region>
                write_triple(region, 'rdf:type', 'SO:five_prime_flanking_region')
                # <BNODE:motif_set><rdfs:label><motif_set_sig>
                write_triple(motifset, 'rdfs:label', motifsetsig)
                # <BNODE:motif_set><rdf:type><SIO:collection>
                write_triple(motifset, 'rdf:type', 'SIO:collection')

In [7]:
len(triples)

248994

In [8]:
print(triples[1])
print(triples[len(triples)-1])

_:b4e88dc925d53f9f5b13 rdfs:label NCBIGene:100037417_up2k_@+24308525 .
_:bab65abf77bc97165a69 rdf:type SIO:collection .


splitting the motifset count out b/c it is independent of the gene/regions

    # <BNODE:motif_set><rdf:value><3>
    
in `motifsetsig_count.tab`

In [9]:
with open('motifsetsig_count.tab', 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:
        line_counter += 1     
        (motifsetsig, count) = row
        motifset = '_:' + digest_id(motifsetsig)
        # <BNODE:motif_set><rdf:value><3>
        write_triple(motifset, 'rdf:value', count)

In [10]:
print(len(triples))
print(triples[len(triples)-1])

254994
_:b6589fc6ab0dc82cf120 rdf:value -2 .


    # <BNODE:motif_set><OIO:subset><BNODE:motif_set>

comes from `motifsetsig_subset.tab`

In [11]:
with open('motifsetsig_subset.tab', 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:     
        (motifsetsig, subsetsig) = row
        motifset = '_:' + digest_id(motifsetsig)
        subset = '_:' + digest_id(subsetsig)
        # <BNODE:motif_set><OIO:subset><BNODE:motif_set>
        write_triple(motifset, 'OIO:subset', subset)

In [12]:
print(len(triples))
print(triples[len(triples)-1])

265430
_:b1d5781111d84f7b3fe4 OIO:subset _:ba4b9237bacccdf19c07 .


    # <BNODE:motif_set><OIO:hasdbxref><http:JASPAR:motif>
    # <BNODE:motif_set><RO:has member><JASPAR:motif> 
    
will just do the first with data from `motifsetsig_motif.tab`

In [13]:
with open('motifsetsig_motif.tab', 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:     
        (motifsetsig, motif) = row
        motifset = '_:' + digest_id(motifsetsig)
        # <BNODE:motif_set><OIO:hasdbxref><http:JASPAR:motif>
        write_triple(motifset, 'OIO:hasdbxref', "JASPAR:" + motif)

In [14]:
print(len(triples))
print(triples[len(triples)-1])

314226
_:b093da02f1d652201da3 OIO:hasdbxref JASPAR:MA0940.1 .


    # <BNODE:gene_upstream_region><RO:member of><BNODE:gene_jaccard_value>
    # <BNODE:gene2_upstream_region><RO:member of><BNODE:gene_jaccard_value>
    
    # <BNODE:gene_jaccard_value><rdfs:label><gene1_region gene2_region>
    # <BNODE:gene_jaccard_value><SWO:Similarity score>	<0.73>
    # <BNODE:gene_jaccard_value><rdf:type><SWO:Jaccard’s index> 
    
 previously created    
    gene = 'NCBIGene:' + str(entrezid)  
    region_label = gene + "_up"  + extent + "_@" + centroid  
    region = '_:' + digest_id(region_label)  
    
 will need   
    (extent)    gene1    gene2    jaccard  
    
 maybe also  
    
    (extent)    gene    dimotifsig  
    
    
    
    
    

In [15]:
dimotif_regions = {    
    '1k' : {
        'f1' : 'gene_pair_dimotif_jaccard_1k.tab'
    }, 
    '2k' : {
        'f1' : 'gene_pair_dimotif_jaccard_2k.tab'
    },
    '5k' : {
        'f1' : 'gene_pair_dimotif_jaccard_5k.tab'
    }
}

for extent in dimotif_regions:
    # print(extent)
    # print(regions[extent])
    for fname in dimotif_regions[extent]:
       # print(fname)
       # print(regions[extent][fname])    
        with open(dimotif_regions[extent][fname], 'r') as tabfile:
            filereader = csv.reader(tabfile, delimiter='\t')
            line_counter = 0
            for row in filereader:
                line_counter += 1     
                (gene1id, centroid1, gene2id, centroid2, jaccard) = row
                gene1 = 'NCBIGene:' + str(gene1id)
                region1_label = gene1 + "_up"  + extent + "_@" + centroid1
                gene2 = 'NCBIGene:' + str(gene1id)
                region2_label = gene2 + "_up"  + extent + "_@" + centroid2
                pairwise_label = region1_label + " & " + region2_label
                pairwise_id =  "_:" + digest_id(pairwise_label)
                # <BNODE:gene_jaccard_value><rdf:type><SWO:Jaccard’s index>
                write_triple(pairwise_id, 'rdf:type', "SWO:Jaccard’s index")
                # <BNODE:gene_jaccard_value><rdfs:label><gene1_region gene2_region>
                write_triple(pairwise_id, 'rdfs:label', pairwise_label)
                # <BNODE:gene_jaccard_value><SWO:Similarity score>	<0.73>
                write_triple(pairwise_id, 'SWO:Similarity score', jaccard) 
                # <BNODE:gene1_upstream_region><RO:member of><BNODE:gene_jaccard_value>
                write_triple ("_:" + digest_id(region1_label), 'RO:member of', pairwise_id)     
                # <BNODE:gene2_upstream_region><RO:member of><BNODE:gene_jaccard_value>
                write_triple ("_:" + digest_id(region2_label), 'RO:member of', pairwise_id)
        print(len(triples))
        print(triples[len(triples)-1])
   
                
                

550541
_:b5ddde0732e9a90a4439 RO:member of _:b30b48bbba03925a46c2 .
687701
_:b9ac6aff8a45db6038b3 RO:member of _:ba67834c51275ad0768a .
1182261
_:b61cfb479592e1975cff RO:member of _:b6a597111b85ff48d8a3 .


In [16]:
fh = open('jaspar_curies.nt', 'w')
print('\n'.join(list(set(triples))), file=fh)
      
fh.close()